In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rc
%matplotlib inline

In [2]:
rc('font',family='Comic Sans MS')

In [19]:
def Z(omega,R,C,L):
    Z = R + 1j*(omega*L - (1/omega*C))
    return Z
def Z_sq(omega,R,C,L):
    ZZ = R**2 + (omega*L - (1/omega*C))**2
    return ZZ

In [22]:
#Szeregowy układ
Cze33 = pd.read_csv(f"./Dane/BodeSimulator33.txt",sep=r'\s*', skiprows=3, header=None, decimal=',')
Cze130 = pd.read_csv(f"./Dane/BodeSimulator130.txt",sep=r'\s*', skiprows=3, header=None, decimal=',')
Cze130.columns = ["Elv_Freq(Hz)","Elv_Gain(dB)","Elv_Phase(deg)","Sym_Freq(Hz)","Sym_Gain(dB)","Sym_Phase(deg)"]
Cze33.columns = ["Elv_Freq(Hz)","Elv_Gain(dB)","Elv_Phase(deg)","Sym_Freq(Hz)","Sym_Gain(dB)","Sym_Phase(deg)"]
Cze33['Elv_Freq(kHz)'] = Cze33['Elv_Freq(Hz)'] / 1e3
Cze130['Elv_Freq(kHz)'] = Cze130['Elv_Freq(Hz)'] / 1e3
Cze33['Elv_Gain'] = np.exp(Cze33['Elv_Gain(dB)'] / 20)
Cze130['Elv_Gain'] = np.exp(Cze130['Elv_Gain(dB)'] / 20)
Cze33['Sym_Freq(kHz)'] = Cze33['Sym_Freq(Hz)'] / 1e3
Cze130['Sym_Freq(kHz)'] = Cze130['Sym_Freq(Hz)'] / 1e3
Cze33['Sym_Gain'] = np.exp(Cze33['Sym_Gain(dB)'] / 20)
Cze130['Sym_Gain'] = np.exp(Cze130['Sym_Gain(dB)'] / 20)

/home/paulina/.anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  
/home/paulina/.anaconda3/lib/python3.6/site-packages/pandas/io/parsers.py:2138: FutureWarning: split() requires a non-empty pattern match.
  yield pat.split(line.strip())
/home/paulina/.anaconda3/lib/python3.6/site-packages/pandas/io/parsers.py:2140: FutureWarning: split() requires a non-empty pattern match.
  yield pat.split(line.strip())
/home/paulina/.anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  This is separate fr